查询pip下载当前所使用的源

In [1]:
pip config list

global.index-url='https://mirror.sjtu.edu.cn/pypi/web/simple/'
Note: you may need to restart the kernel to use updated packages.


##### 换源

常使用的源：<br />
清华：https://pypi.tuna.tsinghua.edu.cn/simple/ <br />
阿里云：http://mirrors.aliyun.com/pypi/simple/ <br />
中国科技大学：https://pypi.mirrors.ustc.edu.cn/simple/ <br />
华中科技大学：http://pypi.hustunique.com/simple/ <br />
上海交通大学：https://mirror.sjtu.edu.cn/pypi/web/simple/ <br />
豆瓣：http://pypi.douban.com/simple/ <br />

In [ ]:
pip config set global.index-url 源地址